In [2]:
import pandas as pd
import numpy as np
import random 

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error

import xgboost
import numerapi
NAPI = numerapi.NumerAPI(verbosity="info")
import random as rn
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from bayes_opt import BayesianOptimization
import os
import pickle

from utils import *
from xgboost_feval import *
from data_prep_qurty import prepare_data
DIR = "data"

In [4]:
download_current_data(DIR)
train_static, val_static, tournament_static = load_data(DIR, reduce_memory=True)
features_list = generate_features_list(train_static)

train_with_group = get_group_stats(train_static)
val_with_group = get_group_stats(val_static)
tournament_with_group = get_group_stats(tournament_static)
ft_corr_list = random.sample(features_list, 31)


You already have the newest data! Current round is: 257
Loading the data


/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [5]:
train, tournament = prepare_data()

You already have the newest data! Current round is: 257
Loading the data


In [8]:
model = train_model(train)

[1]	train-rmse:0.22314	train-pos_corr:0.03784	test-rmse:0.22368	test-pos_corr:0.02953
[2]	train-rmse:0.22312	train-pos_corr:0.04158	test-rmse:0.22367	test-pos_corr:0.03268
[3]	train-rmse:0.22312	train-pos_corr:0.04348	test-rmse:0.22367	test-pos_corr:0.03319
[4]	train-rmse:0.22310	train-pos_corr:0.04574	test-rmse:0.22366	test-pos_corr:0.03489
[5]	train-rmse:0.22309	train-pos_corr:0.04829	test-rmse:0.22365	test-pos_corr:0.03701
[6]	train-rmse:0.22308	train-pos_corr:0.05052	test-rmse:0.22365	test-pos_corr:0.03686
[7]	train-rmse:0.22307	train-pos_corr:0.05125	test-rmse:0.22364	test-pos_corr:0.03724
[8]	train-rmse:0.22307	train-pos_corr:0.05328	test-rmse:0.22364	test-pos_corr:0.03707
[9]	train-rmse:0.22306	train-pos_corr:0.05415	test-rmse:0.22363	test-pos_corr:0.03749
[11]	train-rmse:0.22304	train-pos_corr:0.05419	test-rmse:0.22362	test-pos_corr:0.03737
[12]	train-rmse:0.22304	train-pos_corr:0.05346	test-rmse:0.22362	test-pos_corr:0.03744
[13]	train-rmse:0.22304	train-pos_corr:0.05326	test-

In [9]:
filehandler = open(b"qurty_model_2.pickle.dat","wb")
pickle.dump(model, filehandler)

In [7]:
def train_model(train, model=None):
    X, y = clean_for_xgboost(train)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
    dtrain = xgboost.DMatrix(X_train, y_train)
    dtest = xgboost.DMatrix(X_test, y_test)

    param = {
        'colsample_bytree': 0.7334,
        'gamma': 1.412,
        'max_depth': 10,
        'min_child_weight': 18.26,
        'subsample': 1.0,
        'max_depth':5,
        'eta':0.05,
        'verbosity':0,
        'objective':'reg:squarederror',
        'eval_metric': 'rmse',
        'nthread': -1,
    }
    evals_result = {}
    
    if model is not None:
        tmodel = xgboost.train(
            params=param,
            dtrain=dtrain,
            feval=correlation_score,
            num_boost_round=100,
            evals=[(dtrain, 'train'), (dtest, 'test')],
            evals_result=evals_result,
            xgb_model=model,
            verbose_eval=10,
        )
    else:
        tmodel = xgboost.train(
            params=param,
            dtrain=dtrain,
            feval=correlation_score,
            num_boost_round=100,
            evals=[(dtrain, 'train'), (dtest, 'test')],
            evals_result=evals_result,
            verbose_eval=10,
        )
        
    return tmodel

In [4]:
def train_test_dataframe(df):
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    test = df[~msk]
    return train, test

In [5]:
grouped = train_with_group.groupby('era')
model = None
ft_corr_list = random.sample(features_list, 31)
count = 0 
for name, group in grouped:
    count = count + 1
    if count > 10:
        break
    train, test = train_test_dataframe(group)
    print("Current era", name)
    model = train_model(train, test, ft_corr_list, model)

Current era era1
[1]	train-rmse:0.22324	train-pos_corr:0.04073	test-rmse:0.22348	test-pos_corr:0.01387
[2]	train-rmse:0.22323	train-pos_corr:0.04237	test-rmse:0.22348	test-pos_corr:0.01391
[3]	train-rmse:0.22323	train-pos_corr:0.04416	test-rmse:0.22348	test-pos_corr:0.01472
[4]	train-rmse:0.22321	train-pos_corr:0.04566	test-rmse:0.22347	test-pos_corr:0.01606
[5]	train-rmse:0.22320	train-pos_corr:0.04824	test-rmse:0.22347	test-pos_corr:0.01759
[6]	train-rmse:0.22319	train-pos_corr:0.04845	test-rmse:0.22346	test-pos_corr:0.01740
[7]	train-rmse:0.22318	train-pos_corr:0.04949	test-rmse:0.22346	test-pos_corr:0.01718
[8]	train-rmse:0.22317	train-pos_corr:0.05026	test-rmse:0.22346	test-pos_corr:0.01755
[9]	train-rmse:0.22316	train-pos_corr:0.05160	test-rmse:0.22346	test-pos_corr:0.01859
[11]	train-rmse:0.22315	train-pos_corr:0.05327	test-rmse:0.22345	test-pos_corr:0.01831
[12]	train-rmse:0.22314	train-pos_corr:0.05398	test-rmse:0.22345	test-pos_corr:0.01821
[13]	train-rmse:0.22314	train-pos_c

[5]	train-rmse:0.22296	train-pos_corr:0.06770	test-rmse:0.22344	test-pos_corr:0.02040
[6]	train-rmse:0.22296	train-pos_corr:0.06785	test-rmse:0.22344	test-pos_corr:0.02033
[7]	train-rmse:0.22296	train-pos_corr:0.06785	test-rmse:0.22344	test-pos_corr:0.02033
[8]	train-rmse:0.22296	train-pos_corr:0.06785	test-rmse:0.22344	test-pos_corr:0.02033
[9]	train-rmse:0.22296	train-pos_corr:0.06785	test-rmse:0.22344	test-pos_corr:0.02033
[11]	train-rmse:0.22296	train-pos_corr:0.06785	test-rmse:0.22344	test-pos_corr:0.02033
[12]	train-rmse:0.22296	train-pos_corr:0.06805	test-rmse:0.22344	test-pos_corr:0.02039
[13]	train-rmse:0.22296	train-pos_corr:0.06805	test-rmse:0.22344	test-pos_corr:0.02039
[14]	train-rmse:0.22296	train-pos_corr:0.06821	test-rmse:0.22344	test-pos_corr:0.02058
[15]	train-rmse:0.22296	train-pos_corr:0.06821	test-rmse:0.22344	test-pos_corr:0.02058
[16]	train-rmse:0.22296	train-pos_corr:0.06853	test-rmse:0.22344	test-pos_corr:0.02065
[17]	train-rmse:0.22296	train-pos_corr:0.06853	t

[11]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[12]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[13]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[14]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[15]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[16]	train-rmse:0.22293	train-pos_corr:0.07086	test-rmse:0.22344	test-pos_corr:0.02118
[17]	train-rmse:0.22292	train-pos_corr:0.07101	test-rmse:0.22344	test-pos_corr:0.02138
[18]	train-rmse:0.22292	train-pos_corr:0.07101	test-rmse:0.22344	test-pos_corr:0.02138
[19]	train-rmse:0.22292	train-pos_corr:0.07131	test-rmse:0.22344	test-pos_corr:0.02140
[21]	train-rmse:0.22292	train-pos_corr:0.07145	test-rmse:0.22344	test-pos_corr:0.02150
[22]	train-rmse:0.22292	train-pos_corr:0.07145	test-rmse:0.22344	test-pos_corr:0.02150
[23]	train-rmse:0.22292	train-pos_corr:0.07

[16]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[17]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[18]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[19]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[21]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[22]	train-rmse:0.22290	train-pos_corr:0.07344	test-rmse:0.22344	test-pos_corr:0.02155
[23]	train-rmse:0.22289	train-pos_corr:0.07348	test-rmse:0.22344	test-pos_corr:0.02159
[24]	train-rmse:0.22289	train-pos_corr:0.07348	test-rmse:0.22344	test-pos_corr:0.02159
[25]	train-rmse:0.22289	train-pos_corr:0.07348	test-rmse:0.22344	test-pos_corr:0.02159
[26]	train-rmse:0.22289	train-pos_corr:0.07348	test-rmse:0.22344	test-pos_corr:0.02159
[27]	train-rmse:0.22289	train-pos_corr:0.07348	test-rmse:0.22344	test-pos_corr:0.02159
[28]	train-rmse:0.22289	train-pos_corr:0.07

[22]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[23]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[24]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[25]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[26]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[27]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[28]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[29]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[31]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[32]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[33]	train-rmse:0.22287	train-pos_corr:0.07481	test-rmse:0.22344	test-pos_corr:0.02172
[34]	train-rmse:0.22287	train-pos_corr:0.07

[27]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[28]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[29]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[31]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[32]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[33]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[34]	train-rmse:0.22284	train-pos_corr:0.07749	test-rmse:0.22344	test-pos_corr:0.02249
[35]	train-rmse:0.22284	train-pos_corr:0.07772	test-rmse:0.22344	test-pos_corr:0.02253
[36]	train-rmse:0.22284	train-pos_corr:0.07772	test-rmse:0.22344	test-pos_corr:0.02253
[37]	train-rmse:0.22284	train-pos_corr:0.07791	test-rmse:0.22344	test-pos_corr:0.02236
[38]	train-rmse:0.22284	train-pos_corr:0.07791	test-rmse:0.22344	test-pos_corr:0.02236
[39]	train-rmse:0.22284	train-pos_corr:0.07

[33]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[34]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[35]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[36]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[37]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[38]	train-rmse:0.22283	train-pos_corr:0.07861	test-rmse:0.22344	test-pos_corr:0.02264
[39]	train-rmse:0.22283	train-pos_corr:0.07881	test-rmse:0.22344	test-pos_corr:0.02269
[41]	train-rmse:0.22283	train-pos_corr:0.07881	test-rmse:0.22344	test-pos_corr:0.02269
[42]	train-rmse:0.22283	train-pos_corr:0.07881	test-rmse:0.22344	test-pos_corr:0.02269
[43]	train-rmse:0.22283	train-pos_corr:0.07881	test-rmse:0.22344	test-pos_corr:0.02269
[44]	train-rmse:0.22283	train-pos_corr:0.07881	test-rmse:0.22344	test-pos_corr:0.02269
[45]	train-rmse:0.22283	train-pos_corr:0.07

[38]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[39]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[41]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[42]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[43]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[44]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[45]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[46]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[47]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[48]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[49]	train-rmse:0.22280	train-pos_corr:0.08066	test-rmse:0.22344	test-pos_corr:0.02315
[51]	train-rmse:0.22280	train-pos_corr:0.08

[44]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[45]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[46]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[47]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[48]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[49]	train-rmse:0.22280	train-pos_corr:0.08101	test-rmse:0.22344	test-pos_corr:0.02315
[51]	train-rmse:0.22280	train-pos_corr:0.08121	test-rmse:0.22344	test-pos_corr:0.02322
[52]	train-rmse:0.22280	train-pos_corr:0.08121	test-rmse:0.22344	test-pos_corr:0.02322
[53]	train-rmse:0.22280	train-pos_corr:0.08121	test-rmse:0.22344	test-pos_corr:0.02322
[54]	train-rmse:0.22280	train-pos_corr:0.08121	test-rmse:0.22344	test-pos_corr:0.02322
[55]	train-rmse:0.22280	train-pos_corr:0.08121	test-rmse:0.22344	test-pos_corr:0.02322
[56]	train-rmse:0.22280	train-pos_corr:0.08

[49]	train-rmse:0.22279	train-pos_corr:0.08157	test-rmse:0.22344	test-pos_corr:0.02307
[51]	train-rmse:0.22279	train-pos_corr:0.08157	test-rmse:0.22344	test-pos_corr:0.02307
[52]	train-rmse:0.22279	train-pos_corr:0.08157	test-rmse:0.22344	test-pos_corr:0.02307
[53]	train-rmse:0.22279	train-pos_corr:0.08157	test-rmse:0.22344	test-pos_corr:0.02307
[54]	train-rmse:0.22279	train-pos_corr:0.08157	test-rmse:0.22344	test-pos_corr:0.02307
[55]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[56]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[57]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[58]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[59]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[61]	train-rmse:0.22279	train-pos_corr:0.08178	test-rmse:0.22344	test-pos_corr:0.02311
[62]	train-rmse:0.22279	train-pos_corr:0.08

In [12]:
train, val, tournament = generate_polynomial_features(ft_corr_list,
                                                     (train_with_group, val_with_group, tournament_with_group))

In [10]:
save_model(model, f'model_disc.pickle.dat')

In [9]:
ft_corr_list
import pickle
with open('disc_features1.pkl', 'wb') as f:
    pickle.dump(ft_corr_list, f)

In [13]:
submissions = []
X_tournament, y_tournament = clean_for_xgboost(tournament)
dtournament = xgboost.DMatrix(X_tournament, y_tournament)

tournament.loc[:,"prediction"] = model.predict(dtournament)
submissions.append(tournament)
generate_submission(tournament, name=f'model_disc_submission.csv')

In [ ]:
ft_corr_list = random.sample(features_list, 31)
train, val, tournament = generate_polynomial_features(ft_corr_list,
                                                     (train_with_group, val_with_group, tournament_with_group))

X_train, y_train = clean_for_xgboost(train)
X_val, y_val = clean_for_xgboost(val)
# X_tournament, y_tournament = clean_for_xgboost(tournament)

dtrain = xgboost.DMatrix(X_train, y_train)
dtest = xgboost.DMatrix(X_val, y_val)
# dtournament = xgboost.DMatrix(X_tournament, y_tournament)

param = {
    'max_depth':5,
    'eta':0.05,
    'verbosity':0,
    'objective':'reg:logistic',
    'nthread': -1,
}
evals_result = {}
model = xgboost.train(
    params=param,
    dtrain=dtrain,
    feval=corr_score,
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    evals_result=evals_result,
    verbose_eval=10,
)
model_name = 'hello'
save_model(model, f'model_{model_name}.pickle.dat')

In [ ]:
tournament = tournament_with_group
X_tournament, y_tournament = clean_for_xgboost(tournament)
dtournament = xgboost.DMatrix(X_tournament, y_tournament)
# model = load_model("models/qurty/model_qurty.pickle.dat")
tournament.loc[:,"prediction"] = model.predict(dtournament)
submissions.append(tournament)
generate_submission(tournament, name=f'model_qurty_submission.csv')

In [ ]:
def neutralize(series, by, proportion):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized

In [ ]:
by = pd.read_csv('data/numerai_dataset_'+str(NAPI.get_current_round())+'/example_predictions.csv')

neut = pd.DataFrame({'prediction': neutralize(tournament['prediction'], by['prediction'], 0.3)})

conc=pd.concat([by.drop(columns="prediction"),neut], axis=1)
conc.to_csv("neutralized_submission_file.csv", index=False) # submission file

extra example code

In [ ]:
ft_corr_list = random.sample(features_list, 31)
train, val, tournament = generate_polynomial_features(ft_corr_list,
                                                     (train_with_group, val_with_group, tournament_with_group))

X_train, y_train = clean_for_xgboost(train)
X_val, y_val = clean_for_xgboost(val)
# X_tournament, y_tournament = clean_for_xgboost(tournament)

dtrain = xgboost.DMatrix(X_train, y_train)
dtest = xgboost.DMatrix(X_val, y_val)
# dtournament = xgboost.DMatrix(X_tournament, y_tournament)

param = {
    'max_depth':5,
    'eta':0.05,
    'verbosity':0,
    'objective':'reg:logistic',
    'nthread': -1,
}
evals_result = {}
model = xgboost.train(
    params=param,
    dtrain=dtrain,
    feval=corr_score,
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    evals_result=evals_result,
    verbose_eval=10,
)
model_name = 'hello'
save_model(model, f'model_{model_name}.pickle.dat')